In [ ]:
#import necessary libraries
import numpy as np
from itertools import permutations
import time
import warnings
warnings.filterwarnings("ignore")

#define Graph
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'S']
edges = [('A', 'B', 2), ('A', 'C', 1), ('B', 'F', 1), ('B', 'G', 1),
         ('C', 'D', 2), ('C', 'E', 2), ('D', 'E', 1), ('D', 'K', 1),
         ('E', 'F', 3), ('E', 'S', 2), ('F', 'G', 1), ('G', 'H', 1),
         ('H', 'S', 1), ('I', 'A', 2), ('J', 'C', 1)]
IDA_tsp_time_complexity, IDA_tsp_space_complexity = 0, 0
#calulate shortest path using IDA* Algorithm
def heuristic_shortest_path_ida(node, goal):
    return abs(ord(node) - ord(goal))

def ida_shortest_path(vertices, edges, start_node, goal_node, heuristic_shortest_path_ida):
    threshold = 0
    visited = set()
    path = []
    f_scores = {start_node: heuristic_shortest_path_ida(start_node, goal_node)}
    nodes_expanded = [0]  # Using a list to pass by reference
    max_depth = [0]  # Using a list to pass by reference

    start_time = time.time()  # Start timer

    while True:
        result = search(vertices, edges, start_node, goal_node, threshold, visited, path, f_scores, nodes_expanded, max_depth)
        if result is not None:
            end_time = time.time()  # End timer
            execution_time = end_time - start_time
            return result[0], result[1], execution_time, nodes_expanded[0], max_depth[0]
        threshold += 1

def search_shortest_path_ida(vertices, edges, node, goal_node, threshold, visited, path, f_scores, nodes_expanded, max_depth):
    visited.add(node)
    path.append(node)
    max_depth[0] = max(max_depth[0], len(path))

    if node == goal_node:
        return path, f_scores[node]

    for u, v, w in edges:
        if u == node:
            neighbor = v
            weight = w
        elif v == node:
            neighbor = u
            weight = w
        else:
            continue

        if neighbor in visited:
            continue
        nodes_expanded[0] += 1
        tentative_g_score = f_scores[node] + weight
        if tentative_g_score > threshold:
            continue
        f_scores[neighbor] = min(f_scores.get(neighbor, float('inf')), tentative_g_score + heuristic_shortest_path_ida(neighbor, goal_node))
        result = search_shortest_path_ida(vertices, edges, neighbor, goal_node, threshold, visited.copy(), path[:], f_scores, nodes_expanded, max_depth)
        if result is not None:
            return result

    path.pop()
    return None

# Find the optimal path and cost
start_node = 'A'
goal_node = 'S'
optimal_path, cost, execution_time, nodes_expanded, max_depth = ida_shortest_path(vertices, edges, start_node, goal_node, heuristic_shortest_path_ida)
print("*"*80)
print("Shortes Route from start node to goal node using IDA* algorithm")
print("*"*80)
print(f"Optimal path: {optimal_path}")
print(f"Cost: {cost}")
print(f"Execution time: {execution_time} seconds")
print(f"Space complexity: {nodes_expanded * max_depth} - Nodes expanded: {nodes_expanded}, Maximum depth reached: {max_depth}")


#Route for inspection and maintnance using IDA* Algorithm
def euclidean_distance(node1, node2):
    mask = np.isfinite(node1) & np.isfinite(node2)
    squared_diff = (node1 - node2) ** 2
    squared_diff[~mask] = 0
    euclidean_dist = np.sqrt(np.sum(squared_diff))
    return euclidean_dist

def total_distance(path, distances):
    total = 0
    for i in range(len(path) - 1):
        total += distances[path[i]][path[i+1]]
    total += distances[path[-1]][path[0]]  # return to starting node
    return total

def minimum_spanning_tree(distances):
    num_cities = len(distances)
    visited = [False] * num_cities
    visited[0] = True
    min_tree = []
    edges = []
    for i in range(1, num_cities):
        edges.append((distances[0][i], 0, i))
    edges.sort()
    while len(min_tree) < num_cities - 1:
        min_dist, u, v = edges.pop(0)
        if not visited[v]:
            visited[v] = True
            min_tree.append((u, v))
            for i in range(num_cities):
                if not visited[i]:
                    edges.append((distances[v][i], v, i))
            edges.sort()
    return min_tree

def min_mst_heuristic(min_tree, path, distances):
    remaining_edges = min_tree[:]
    for i in range(len(path) - 1):
        edge = (path[i], path[i+1])
        if edge in remaining_edges:
            remaining_edges.remove(edge)
    if len(path) > 0:
        return_to_start_edge = (path[-1], path[0])
        if return_to_start_edge in remaining_edges:
            remaining_edges.remove(return_to_start_edge)  # return to starting node
    min_spanning_tree_cost = sum(distances[edge[0]][edge[1]] for edge in min_tree)
    min_heuristic = 0
    for u, v in remaining_edges:
        min_heuristic += min(distances[u][i] for i in path) + min(distances[v][i] for i in path)
    return min_spanning_tree_cost + min_heuristic / 2

def create_distance_matrix(vertices, edges):
    index_map = {vertex: index for index, vertex in enumerate(vertices)}
    matrix = np.full((len(vertices), len(vertices)), np.inf)

    for vertex, neighbor, weight in edges:
        matrix[index_map[vertex]][index_map[neighbor]] = weight
        matrix[index_map[neighbor]][index_map[vertex]] = weight

    for i in range(len(vertices)):
        matrix[i][i] = 0

    return matrix

def index_to_vertices(index_path, vertices):
    return [vertices[i] for i in index_path]

def dfs(current, path, min_tree, distances, bound, max_depth, nodes_expanded):
    if len(path) == len(distances):  # Checking path completion
        return path, total_distance(path, distances)  # O(V) time

    min_cost = float('inf')
    min_path = None
    for neighbor in range(len(distances)):  # O(V) iterations
        if neighbor not in path:  # O(V) lookup
            new_path = path + [neighbor]  # Space: O(V)
            cost = total_distance(new_path, distances)  # O(V) time
            if cost <= bound:
                nodes_expanded[0] += 1  # Increment nodes expanded
                result_path, result_cost = dfs(neighbor, new_path, min_tree, distances, bound, max_depth, nodes_expanded)  # Recursive call
                if result_cost < min_cost:
                    min_cost = result_cost
                    min_path = result_path
                    bound = min(bound, min_cost + min_mst_heuristic(min_tree, new_path, distances))  # O(V^2) time
    max_depth[0] = max(max_depth[0], len(path))  # Update max depth
    return min_path, min_cost

def ida_star_tsp(distances):
    min_tree = minimum_spanning_tree(distances)  # O(V^2 log V)
    path = [0]  # Start with node 0
    bound = min_mst_heuristic(min_tree, path, distances)  # O(V^2) time
    max_depth = [0]  # Initialize max depth
    nodes_expanded = [0]  # Initialize nodes expanded
    while True:
        result_path, result_cost = dfs(0, path, min_tree, distances, bound, max_depth, nodes_expanded)  # O(V!) time
        if result_cost == float('inf'):
            return "No solution found", max_depth[0], nodes_expanded[0]  # Return max depth and nodes expanded
        if result_cost < bound:
            bound = result_cost
            path = result_path
        else:
            return path, result_cost, max_depth[0], nodes_expanded[0]  # Return max depth and nodes expanded

nodes = create_distance_matrix(vertices, edges)

distances = np.array([[euclidean_distance(node1, node2) for node2 in nodes] for node1 in nodes])

start_time = time.time()  # Start timer
result = ida_star_tsp(distances)
end_time = time.time()  # End timer

if isinstance(result, tuple):
    path, cost, max_depth, nodes_expanded = result
    print("\n"+ "*"*80)
    print("Optimal Route for Inspection and Maintainance using IDA* algorithm")
    print("*"*80)
    print("Optimal Path:", index_to_vertices(path, vertices))
    print("Execution Time:", end_time - start_time, "seconds")
    print(f"Space complexity: {nodes_expanded * max_depth} - Nodes expanded: {nodes_expanded}, Maximum depth reached: {max_depth}")

else:
    print(result)

********************************************************************************
Shortes Route from start node to goal node using IDA* algorithm
********************************************************************************
Optimal path: ['A', 'C', 'D', 'E', 'S']
Cost: 53
Execution time: 0.0007865428924560547 seconds
Space complexity: 2450 - Nodes expanded: 490, Maximum depth reached: 5

********************************************************************************
Optimal Route for Inspection and Maintainance using IDA* algorithm
********************************************************************************
Optimal Path: ['A', 'H', 'B', 'D', 'G', 'C', 'S', 'I', 'E', 'K', 'F', 'J']
Execution Time: 0.5047183036804199 seconds
Space complexity: 316921 - Nodes expanded: 28811, Maximum depth reached: 11


In [ ]:
def shortest_path_IDA(start_node, goal_node, vertices, edges, heuristic_shortest_path_ida):
  global IDA_shortest_path_time_complexity
  global IDA_shortest_path_space_complexity
  optimal_path, cost, execution_time, nodes_expanded, max_depth = ida_shortest_path(vertices, edges, start_node, goal_node, heuristic_shortest_path_ida)
  print("*"*80)
  print("Shortes Route from start node to goal node using IDA* algorithm")
  print("*"*80)
  print(f"Optimal path: {optimal_path}")
  print(f"Cost: {cost}")
  IDA_shortest_path_time_complexity = execution_time
  IDA_shortest_path_space_complexity = nodes_expanded * max_depth
shortest_path_IDA(start_node, goal_node, vertices, edges, heuristic_shortest_path_ida)

********************************************************************************
Shortes Route from start node to goal node using IDA* algorithm
********************************************************************************
Optimal path: ['A', 'C', 'D', 'E', 'S']
Cost: 53


In [ ]:
def tsp_IDA(vertices, edges):
  global IDA_tsp_time_complexity
  global IDA_tsp_space_complexity
  nodes = create_distance_matrix(vertices, edges)

  distances = np.array([[euclidean_distance(node1, node2) for node2 in nodes] for node1 in nodes])

  start_time = time.time()  # Start timer
  result = ida_star_tsp(distances)
  end_time = time.time()  # End timer

  if isinstance(result, tuple):
      path, cost, max_depth, nodes_expanded = result
      print("\n"+ "*"*80)
      print("Optimal Route for Inspection and Maintainance using IDA* algorithm")
      print("*"*80)
      print("Optimal Path:", index_to_vertices(path, vertices))
      IDA_tsp_time_complexity = end_time - start_time
      IDA_tsp_space_complexity = nodes_expanded * max_depth

  else:
      print(result)
tsp_IDA(vertices, edges)
print(IDA_tsp_time_complexity, IDA_tsp_space_complexity)


********************************************************************************
Optimal Route for Inspection and Maintainance using IDA* algorithm
********************************************************************************
Optimal Path: ['A', 'H', 'B', 'D', 'G', 'C', 'S', 'I', 'E', 'K', 'F', 'J']
0.5168895721435547 316921


In [ ]:
def IDA_complexity():
  print(f"Time complexity IDA* Algoritm for TSP: {IDA_tsp_time_complexity}")
  print(f"Space complexity IDA* Algoritm for TSP: {IDA_tsp_space_complexity}")
  print(f"Time complexity IDA* Algoritm for finding shortest path: {IDA_shortest_path_time_complexity}")
  print(f"Space complexity IDA* Algoritm for finding shortest path: {IDA_shortest_path_space_complexity}")
IDA_complexity()

Time complexity IDA* Algoritm for TSP: 0.5168895721435547
Space complexity IDA* Algoritm for TSP: 316921
Time complexity IDA* Algoritm for finding shortest path: 0.005912303924560547
Space complexity IDA* Algoritm for finding shortest path: 2450


In [ ]:
import numpy as np
import time

# Define the distance function
def euclidean_distance(node1, node2):
    mask = np.isfinite(node1) & np.isfinite(node2)
    squared_diff = (node1 - node2) ** 2
    squared_diff[~mask] = 0
    euclidean_dist = np.sqrt(np.sum(squared_diff))
    return euclidean_dist

# Define the total distance of the path
def total_distance(path, distances):
    total = 0
    for i in range(len(path) - 1):
        total += distances[path[i]][path[i+1]]
    total += distances[path[-1]][path[0]]  # return to starting node
    return total

# Define the hill climbing algorithm
def hill_climbing_tsp(distances, max_iterations=10000):
    num_cities = len(distances)
    current_path = np.random.permutation(num_cities)
    current_cost = total_distance(current_path, distances)
    best_path = current_path.copy()
    best_cost = current_cost
    iterations = 0

    start_time = time.time()  # Start timer

    while iterations < max_iterations:
        new_path = current_path.copy()
        # Perform swap mutation
        idx1, idx2 = np.random.choice(num_cities, 2, replace=False)
        new_path[idx1], new_path[idx2] = new_path[idx2], new_path[idx1]
        new_cost = total_distance(new_path, distances)
        if new_cost < current_cost:
            current_path = new_path
            current_cost = new_cost
            if new_cost < best_cost:
                best_path = new_path
                best_cost = new_cost
        iterations += 1

    end_time = time.time()  # End timer

    return best_path, best_cost, end_time - start_time

# Define vertices and edges
vertices = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'S']
edges = [('A', 'B', 2), ('A', 'C', 1), ('B', 'F', 1), ('B', 'G', 1),
         ('C', 'D', 2), ('C', 'E', 2), ('D', 'E', 1), ('D', 'K', 1),
         ('E', 'F', 3), ('E', 'S', 2), ('F', 'G', 1), ('G', 'H', 1),
         ('H', 'S', 1), ('I', 'A', 2), ('J', 'C', 1)]

# Create distance matrix
def create_distance_matrix(vertices, edges):
    index_map = {vertex: index for index, vertex in enumerate(vertices)}
    matrix = np.full((len(vertices), len(vertices)), np.inf)

    for vertex, neighbor, weight in edges:
        matrix[index_map[vertex]][index_map[neighbor]] = weight
        matrix[index_map[neighbor]][index_map[vertex]] = weight

    for i in range(len(vertices)):
        matrix[i][i] = 0

    return matrix

nodes = create_distance_matrix(vertices, edges)

distances = np.array([[euclidean_distance(node1, node2) for node2 in nodes] for node1 in nodes])

# Run hill climbing algorithm
best_path, best_cost, execution_time = hill_climbing_tsp(distances)

# Output results
print("*"*80)
print("Optimal Route for Inspection and Maintenance using Hill Climbing algorithm")
print("*"*80)
print("Optimal Path:", [vertices[i] for i in best_path])
print("Optimal Cost:", best_cost)
print("Execution Time:", execution_time, "seconds")


********************************************************************************
Optimal Route for Inspection and Maintenance using Hill Climbing algorithm
********************************************************************************
Optimal Path: ['I', 'F', 'H', 'A', 'J', 'S', 'B', 'D', 'G', 'C', 'K', 'E']
Optimal Cost: 1.0
Execution Time: 0.25408005714416504 seconds


In [ ]:
def hill_climbing_shortest_path(vertices, edges, start_node, goal_node, heuristic_shortest_path_hill_climbing):
    current_node = start_node
    path = [current_node]
    total_cost = 0

    while current_node != goal_node:
        neighbors = [(v, w) for u, v, w in edges if u == current_node or v == current_node]
        next_node, min_cost = min(neighbors, key=lambda x: x[1] + heuristic_shortest_path_hill_climbing(x[0], goal_node))
        path.append(next_node)
        total_cost += min_cost
        current_node = next_node

    return path, total_cost

# Calculate shortest path using Hill Climbing
start_time = time.time()  # Start timer
hill_climbing_path, hill_climbing_cost = hill_climbing_shortest_path(vertices, edges, start_node, goal_node, heuristic_shortest_path_ida)
end_time = time.time()  # End timer

print("\n" + "*"*80)
print("Shortest Route from start node to goal node using Hill Climbing algorithm")
print("*"*80)
print("Optimal Path:", hill_climbing_path)
print("Cost:", hill_climbing_cost)
print("Execution Time:", end_time - start_time, "seconds")



********************************************************************************
Shortest Route from start node to goal node using Hill Climbing algorithm
********************************************************************************
Optimal Path: ['A', 'C', 'E', 'S']
Cost: 5
Execution Time: 9.322166442871094e-05 seconds
